In [12]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import os
import numpy as np

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Data4') 

# Actions that we try to detect
actions = np.array(['come', 'left', 'right'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [10]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'come': 0, 'left': 1, 'right': 2}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [62]:
np.save('y_test.npy', y_test)
np.save('x_test.npy', x_test)

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0]])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [27]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(72, 30, 126)
(72, 3)
(18, 30, 126)
(18, 3)


In [ ]:
# Converting trained model to tflite and using the script for inference on edge
import tensorflow as tf
import numpy as np

# class TestModel(tf.Module):
#       def __init__(self):
#         super(TestModel, self).__init__()

#       @tf.function(input_signature=[tflite.TensorSpec(shape=[1, 10], dtype=tflite.float32)])
#       def add(self, x):
#         '''
#         Simple method that accepts single input 'x' and returns 'x' + 4.
#         '''
#         # Name the output 'result' for convenience.
#         return {'result' : x + 4}


# SAVED_MODEL_PATH = 'content/saved_models/test_variable'
TFLITE_FILE_PATH = 'action3.tflite'

# # Save the model
# module = TestModel()
# # You can omit the signatures argument and a default signature name will be
# # created with name 'serving_default'.
# tflite.saved_model.save(
#     module, SAVED_MODEL_PATH,
#     signatures={'my_signature':module.add.get_concrete_function()})

# # Convert the model using TFLiteConverter
# converter = tflite.TFLiteConverter.from_saved_model(SAVED_MODEL_PATH)
# tflite_model = converter.convert()
# with open(TFLITE_FILE_PATH, 'wb') as f:
#   f.write(tflite_model)

# Load the TFLite model in TFLite Interpreter
interpreter = tf.lite.Interpreter(TFLITE_FILE_PATH)
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# print(output_details)
# print()
# print(input_details)

# Test the model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

# There is only 1 signature defined in the model,
# so it will return it by default.
# If there are multiple signatures then we can pass the name.
# my_signature = interpreter.get_signature_runner()

# # my_signature is callable with input as arguments.
# output = my_signature(x=tflite.constant([1.0], shape=(1,10), dtype=tflite.float32))
# # 'output' is dictionary with all outputs from the inference.
# # In this case we have single output 'result'.
# print(output['result']) 

[[0. 1. 0.]]


In [30]:
import tensorflow as tf
from tensorflow import keras

# # Convert the model
# converter = tf.lite.TFLiteConverter.from_saved_model() # path to the SavedModel directory
# tflite_model = converter.convert()

# # Save the model.
# with open('action_saved.tflite', 'wb') as f:
#   f.write(tflite_model)

model = keras.models.load_model('action5.h5', compile=False)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

# # Save the model.
with open('action5.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Austin\AppData\Local\Temp\tmpusk73c5t\assets


INFO:tensorflow:Assets written to: C:\Users\Austin\AppData\Local\Temp\tmpusk73c5t\assets


In [32]:
import tensorflow as tf
import numpy as np 

# SAVED_MODEL_PATH 
TFLITE_FILE_PATH = 'action5.tflite'

# Loading the tensorflow model
interpreter = tf.lite.Interpreter(model_path=TFLITE_FILE_PATH)

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(output_details)
print()
print(input_details)

[{'name': 'StatefulPartitionedCall:0', 'index': 70, 'shape': array([1, 3]), 'shape_signature': array([-1,  3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

[{'name': 'serving_default_lstm_input:0', 'index': 0, 'shape': array([  1,  30, 126]), 'shape_signature': array([ -1,  30, 126]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [17]:
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
tflite_model_predictions.shape

Input Shape: [  1  30 126]
Input Type: <class 'numpy.float32'>
Output Shape: [1 3]
Output Type: <class 'numpy.float32'>


(1, 3)

In [36]:

interpreter.resize_tensor_input(input_details[0]['index'], (18, 30,126))
interpreter.resize_tensor_input(output_details[0]['index'], (18, 3))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [ 18  30 126]
Input Type: <class 'numpy.float32'>
Output Shape: [18  3]
Output Type: <class 'numpy.float32'>


In [43]:
X_test.shape

(18, 30, 126)

In [44]:
test_imgs_numpy = np.array(X_test, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
tflite_model_predictions


array([[2.21071932e-02, 6.81504243e-05, 9.77824628e-01],
       [2.03051195e-02, 5.56650521e-05, 9.79639173e-01],
       [2.14390699e-02, 6.26597248e-05, 9.78498280e-01],
       [2.37282598e-03, 6.75294432e-04, 9.96951818e-01],
       [6.51056468e-02, 9.34887409e-01, 7.02643456e-06],
       [1.08471380e-04, 9.99891520e-01, 1.73974168e-10],
       [7.02875433e-04, 9.99297142e-01, 5.21005727e-09],
       [2.22675689e-02, 7.06468927e-05, 9.77661788e-01],
       [1.26626506e-03, 5.74907637e-04, 9.98158872e-01],
       [2.65106503e-02, 7.26365397e-05, 9.73416686e-01],
       [9.99826968e-01, 1.17259835e-04, 5.57587664e-05],
       [9.96302485e-01, 1.90531753e-03, 1.79222354e-03],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [3.30125367e-05, 9.99966979e-01, 2.61255479e-11],
       [9.99828577e-01, 8.06925382e-05, 9.06508503e-05],
       [2.88497247e-02, 7.86403616e-05, 9.71071601e-01],
       [3.31819116e-04, 5.35781786e-04, 9.99132454e-01],
       [9.99756157e-01, 1.78203

In [45]:
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)
prediction_classes

ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(tflite_model_predictions, axis=1).tolist()

Prediction results shape: (18, 3)


array([2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 0, 0, 1, 1, 0, 2, 2, 0], dtype=int64)

In [58]:
yhat

[2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 0, 0, 1, 1, 0, 2, 2, 0]

In [60]:
for i in range(18):
    if yhat[i] != prediction_classes[i]:
        print('False')

In [47]:
tflite_model_predictions.shape

(18, 3)

In [51]:
prediction_classes.shape

(18,)

In [50]:
y_test

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0]])

In [52]:

acc = accuracy_score(tflite_model_predictions, y_test)
acc

ValueError: Classification metrics can't handle a mix of continuous-multioutput and multilabel-indicator targets

In [53]:

ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(tflite_model_predictions, axis=1).tolist()

In [56]:
ytrue

[2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 0, 0, 1, 1, 0, 2, 2, 0]

In [57]:
yhat

[2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 0, 0, 1, 1, 0, 2, 2, 0]

In [54]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[14,  0],
        [ 0,  4]],

       [[13,  0],
        [ 0,  5]],

       [[ 9,  0],
        [ 0,  9]]], dtype=int64)

In [55]:
accuracy_score(ytrue, yhat)

1.0